# Supplementary figure for nonlinear OSN responses

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns
import os, json
pj = os.path.join

In [ ]:
do_save_plots = True
# Resources
root_dir = pj("..", "..")
data_folder = pj(root_dir, "results", "for_plots")
data_folder_nl = pj(root_dir, "results", "for_plots", "nonlin_adapt")
panels_folder = "panels/"
params_folder = pj(root_dir, "results", "common_params")

In [ ]:
# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full = np.asarray(json.load(f))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

In [ ]:
# Extra aesthetic parameters for this figure
# Figures slightly less high, to squeeze four rows of plots
plt.rcParams["figure.figsize"] = (plt.rcParams["figure.figsize"][0], 1.6)

# More legend rcParams: make everything smaller by 30 %
plt.rcParams["patch.linewidth"] = 0.75
legend_rc = {"labelspacing":0.5, "handlelength":2.0, "handleheight":0.7, 
             "handletextpad":0.8, "borderaxespad":0.5, "columnspacing":2.0}
for k in legend_rc:
    plt.rcParams["legend."+k] = 0.75 * legend_rc[k]

new_color = "r"
linestyles = ["-", "--", ":", (0, (5, 1, 2, 1)), "-."]
neuron_styles = linestyles + [(0, (1, 2, 1, 2))]

In [ ]:
def l2_norm(vecs, axis=-1):
    r""" Computes l2 norm of vectors stored along the last axis of vecs.
    Args:
        vecs can be either a single vector (1d) or an  arbitrary array of vectors,
            where the last dimension indexes elements of vectors.
        axis (int): which axis to sum along.

    Returns: array of distances of same shape as vecs
        except for the summation axis, removed.
    """
    return np.sqrt(np.sum(vecs**2, axis=axis))

# Main figure panels

# Panel A: OSN response function

In [ ]:
# Load data and fit results for the affinities distribution
with open(pj(data_folder_nl, "si2019_cdf_and_fits.json")) as fp:
    si2019_dict = json.load(fp)
    x_sorted = np.asarray(si2019_dict["x_sorted"])
    ccdf_exp = np.asarray(si2019_dict["ccdf_exp"])
    popt_logtanh = np.asarray([si2019_dict["best_fit"]["logb"], 
                               si2019_dict["best_fit"]["alpha"]])
    pstd_logtanh = np.sqrt(np.asarray([si2019_dict["best_fit"]["logb_cov"], 
                               si2019_dict["best_fit"]["alpha_cov"]]))
del si2019_dict

In [ ]:
def logtanh_ccdf_logb(x, logb, alpha):
    """ CCDF fit on the 1/EC50 = K (affinity) distribution"""
    return np.log10(np.tanh(1.0 / ((10.0**logb) * (x**alpha))))

def inverse_transform_tanhcdf(r, logb, alpha):
    return (10.0**logb * np.arctanh(r))**(-1.0 / alpha)

In [ ]:
def osn_response_curve(c, k, epsil, fmax=1.0):
    """ OSN response function to a single odor
    """
    # Dot products over odors
    kc = k*c
    expeps = np.exp(epsil)
    activs = fmax * kc / (expeps + kc)
    return activs

In [ ]:
# Pick three Ks, show the effect of varying $\epsilon$ on one of them. 
n_k_shown = 30
rgen = np.random.default_rng(0x896b0c32a115dd38d3d41a83690f453)
unit_scale = 5e-4
k_samples = inverse_transform_tanhcdf(rgen.uniform(size=n_k_shown), *popt_logtanh)
k_samples[-1] = 1e5
k_samples[-2] = 1e6
k_samples *= unit_scale
default_epsil = 5.0
modified_epsil = 8.0
c_range = np.geomspace(1e-2, 1e2)
default_curves = [osn_response_curve(c_range, k, default_epsil) for k in k_samples]
modified_curve = osn_response_curve(c_range, k_samples[-1], modified_epsil)

In [ ]:
fig, ax = plt.subplots()
curve_colors = ["grey"]*(n_k_shown-2) + ["xkcd:marine", "tab:blue"]
thick_lw = 1.5
for i, k in enumerate(k_samples-2):
    ax.plot(c_range, default_curves[i], color=curve_colors[i], alpha=0.2)
ax.plot(c_range, default_curves[-2], color=curve_colors[-2], lw=thick_lw)
ax.plot(c_range, default_curves[-1], color=curve_colors[-1], lw=thick_lw*1.25)
ax.plot(c_range, modified_curve, color=curve_colors[-1], ls="--", lw=thick_lw)
k_eps_hi = np.exp(-default_epsil)*k_samples[-2]
k_eps_mid = np.exp(-default_epsil)*k_samples[-1]
k_eps_low = np.exp(-modified_epsil)*k_samples[-1]
ax.annotate(r"$\uparrow K$", xy=(1.0/np.sqrt(k_eps_mid*k_eps_hi), 0.53), 
            ha="center", va="bottom", color=curve_colors[-2])
ax.annotate(r"$\uparrow \epsilon$", xy=(1.0/np.sqrt(k_eps_low*k_eps_mid), 0.53), 
            ha="center", va="bottom", color=curve_colors[-1])
arrowprops = {"width":0.5, "headwidth":3.0, "headlength": 2.5, "color":curve_colors[-2]}
ax.annotate("", xy=(1.2/k_eps_hi, 0.5), xytext=(0.8/k_eps_mid, 0.5), arrowprops=arrowprops)
arrowprops.update({"color":curve_colors[-1]})
ax.annotate("", xy=(0.8/k_eps_low, 0.5), xytext=(1.23/k_eps_mid, 0.5), 
            arrowprops=arrowprops)
ax.set(xlabel="Concentration (scaled)", ylabel="OSN response", xscale="log")
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "figure_nl_osn_response_curve.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel B: distribution of OR type EC50s

In [ ]:
xrange = np.geomspace(x_sorted.min(), x_sorted.max(), 200)
ccdf_fit_log = 10.0**(logtanh_ccdf_logb(xrange, *popt_logtanh))
# Upper and lower fit limits, when b is best +- std, alpha is best -+ std
ccdf_fit_lower = 10.0**(logtanh_ccdf_logb(xrange, *(popt_logtanh+2.0*pstd_logtanh)))
ccdf_fit_upper = 10.0**(logtanh_ccdf_logb(xrange, *(popt_logtanh-2.0*pstd_logtanh)))

fig, ax = plt.subplots()
fit_color = "tab:blue"
ax.plot(x_sorted, ccdf_exp, marker="o", mec="k", mfc="none", ms=3.0, ls="none", mew=0.5, 
        label=r'Si $\it{et\, al.}$, 2019')
ax.plot(xrange, ccdf_fit_log, ls="-", label=r"Fit $\mathrm{tanh}(1/bx^{\alpha})$", color=fit_color)
ax.fill_between(xrange, ccdf_fit_lower, ccdf_fit_upper, color=fit_color, alpha=0.3, 
                label=r"95 % CI ($2\sigma$) ")
ax.set_ylabel(r"Complement. CDF $G_X(x)$")
ax.annotate(r"$\alpha = {:.3f} \pm {:.3f}$".format(popt_logtanh[1], pstd_logtanh[1])
            + "\n" + r"$\log_{10} b = " + r"{:.2f} \pm {:.2f}$".format(popt_logtanh[0], pstd_logtanh[0]) , 
           xy=(x_sorted.min()*0.75, 0.1), ha="left", va="top", fontsize=6)
# Vertical line where we clip
krange = 5e2
clip_lim = krange * 10.0**(-popt_logtanh[0]/popt_logtanh[1])
ax.axvline(clip_lim, ls=":", color="k", alpha=0.7, zorder=0)
ax.set_xscale("log")
ax.legend(frameon=False)
ax.set_xlabel(r"$x = K = 1/\mathrm{EC}_{50}$ (dilution units$^{-1}$)")
ax.set_yscale("log")
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "figure_nl_osn_ccdf_fit.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()


# Former panel C: illustrate curved manifold
Dropped after we split the supplementary figure into a main and a supplementary. 

In [ ]:
# Load samples
with np.load(pj(data_folder_nl, "2d_manifold_nonlinear_osn.npz")) as manifold_data:
    conc_ser = manifold_data["conc_ser"]
    vecs = manifold_data["vecs"]
    bkvecser = manifold_data["bkvecser"]
    plane = manifold_data["plane"]

In [ ]:
# Plot 2D manifold in a 3D slice,
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
where_01 = (conc_ser > 0).astype(bool)
locations = {
    "Both odors": np.all(where_01, axis=1),  #all
    "Odor 0": (where_01[:, 0] & ~where_01[:, 1]),  # 0
    "Odor 1": (~where_01[:, 0] & where_01[:, 1])  # 1
}
all_colors = {
    "Both odors": "xkcd:purple",
    "Odor 0": "xkcd:blue",
    "Odor 1": "xkcd:red",

}

# Background odors plane
orig = np.zeros([3, 2])

#topvecs = np.linalg.svd(optim_p)[0][:, :2] * -1
x, y, z = plane[0, ::4], plane[1, ::4], plane[2, ::4]
ls = mpl.colors.LightSource(170, 45)
rgb = ls.shade(z, cmap=mpl.cm.Greys, vert_exag=0.1, blend_mode='soft')
#ax.plot_surface(x, y, z-0.05, color="grey", alpha=0.3, linewidth=0.5, lightsource=ls)
ax.plot_surface(x, y, z-0.05, color="grey", alpha=0.3, rstride=1, cstride=1, facecolors=rgb,
                       linewidth=0, antialiased=False, shade=True)

for lbl in ["Odor 0", "Odor 1", "Both odors"]:
    slc = locations[lbl]
    skp_loc = 3 if lbl.startswith("Both") else 20
    ax.scatter(bkvecser[slc, 0][::skp_loc], bkvecser[slc, 1][::skp_loc], 
               bkvecser[slc, 2][::skp_loc], s=4, lw=0.3, label=lbl, color=all_colors[lbl])

ax.quiver(*orig, *(vecs[:, :3].T), color="k", lw=1.5, arrow_length_ratio=0.2)
ax.scatter(0, 0, 0, color="k", s=25)

# Labeling
for lbl, f in enumerate([ax.set_xlabel, ax.set_ylabel, ax.set_zlabel]):
    # z axis label gets caught in variable zlbl the last iteration
    zlbl = f("OSN {}".format(lbl+1), labelpad=-17.5)
for f in [ax.set_xticks, ax.set_yticks, ax.set_zticks]:
    f([])
for f in [ax.set_xticklabels, ax.set_yticklabels, ax.set_zticklabels]:
    f([], pad=0.1)

ax.view_init(azim=240, elev=35)

leg = ax.legend(frameon=False, loc="upper right", title="Background", ncol=2, title_fontsize=6)
#loc="upper right", bbox_to_anchor=(0.0, 1.0), frameon=False)
fig.tight_layout()

# Need to adjust the tightbox to remove whitespace above and below manually. 
#ax.set_aspect("equal")
fig.tight_layout()
tightbox = fig.get_tightbbox()
tightbox._bbox.y0 = tightbox._bbox.y0*1.3   #bottom
tightbox._bbox.y1 = tightbox._bbox.y1 + 0.3*tightbox._bbox.y0  # bottom
tightbox._bbox.x0 = tightbox._bbox.x0 * 0.4  # position of left side

figname = "supfig_nl_osn_2d_manifold_example.pdf"
if do_save_plots:
    fig.savefig(pj(panels_folder, figname), 
                transparent=True, bbox_inches=tightbox, bbox_extra_artists=(zlbl, leg))
plt.show()
plt.close()

# Panel C: curved manifold at intermediate nonlinearity ($\epsilon = 6.0$)

In [ ]:
# Functions to load simulations results for a given epsilon from within a larger npz archive
def load_ibcm_epsil_simul(fp, epsil):
    epsil = str(epsil)
    simul = {
        "cbars_gamma_ser": fp.get("cbars_gamma_ser_{}".format(epsil)),
        "bkvec_ser": fp.get("bkvec_ser_{}".format(epsil)),
        "back_components": fp.get("back_components_{}".format(epsil)),
        "y_norm_ser": fp.get("y_norm_ser_{}".format(epsil)),
        "conc_ser": fp.get("conc_ser_{}".format(epsil)),
        "mixed_new_odors": fp.get("mixed_new_odors_{}".format(epsil)),
        "new_odors": fp.get("new_odors_{}".format(epsil))
    }
    return simul

def load_biopca_epsil_simul(fp, epsil):
    epsil = str(epsil)
    simul = {
        "true_pca_vals": fp.get("true_pca_vals_{}".format(epsil)),
        "learnt_pca_vals": fp.get("learnt_pca_vals_{}".format(epsil)),
        "pca_align_error": fp.get("pca_align_error_{}".format(epsil)),
        "bkvec_norm_ser": fp.get("bkvec_norm_ser_{}".format(epsil)),
        "y_norm_ser": fp.get("y_norm_ser_{}".format(epsil))
    }
    return simul

In [ ]:
# Since we will make similar plots, define functions
def plot_manifold(bkser, bkvecs, conc_ser, view_params, 
                  mixed_new_odors=None, new_odor_vec=None, dims=(0, 1, 2)):
    # Plot 2D manifold in a 3D slice,
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    # Too many combinations for 6 odors, maybe just highlight
    # single-odor axes
    where_each = (conc_ser > 0).astype(bool)
    n_odors = where_each.shape[1]
    locations = {}
    # Track places with 0 or 1 odor
    any_single_odor = np.all(where_each == False, axis=1)  # start with places with 0 odor
    for i in range(n_odors):
        mask = np.zeros((1, n_odors), dtype=bool)
        mask[0, i] = True
        locations["Odor {}".format(i)] = np.all(where_each == mask, axis=1)
        any_single_odor += locations["Odor {}".format(i)]  # add places with odor i only
    locations["2+ odors"] = np.logical_not(any_single_odor)  # 2+ odors anywhere else
    single_odor_colors = sns.color_palette("colorblind", n_colors=n_odors)
    all_colors = {"Odor {}".format(i): single_odor_colors[i] for i in range(n_odors)}
    all_colors["2+ odors"] = "grey"
    
    orig = np.zeros([3, 6])
    locations_order = ["2+ odors"] + ["Odor {}".format(i) for i in range(n_odors)]
    for lbl in locations_order:
        alpha = 0.3 if lbl.startswith("2+") else 1.0
        slc = locations[lbl]
        tskp = 5 if lbl.startswith("2+") else 1
        zshift = 0.03 if lbl.startswith("2+") else 0.0
        shift = 0.03 if lbl.startswith("2+") else 0.0
        lbl_append = ""# if lbl.startswith("2+") else " alone"
        bk_subset = [bkser[slc, d].copy() + shift for d in dims]
        bk_subset[2] -= zshift
        ax.scatter(bk_subset[0][::tskp], bk_subset[1][::tskp], bk_subset[2][::tskp], 
                   s=4, lw=0.3, label=lbl+lbl_append, color=all_colors[lbl], alpha=alpha)
    vecs = bkvecs / l2_norm(bkvecs, axis=1)[:, None]
    print(vecs.shape)
    ax.quiver(*orig, *(vecs[:, dims].T), color="k", lw=1.5, arrow_length_ratio=0.2)
    ax.scatter(0, 0, 0, color="k", s=25)
    
    # Also show what adding a new odor can do -- out of the manifold?
    new_odor_lbl = "+ new odor"
    if mixed_new_odors is not None:
        n_new_odors = mixed_new_odors.shape[0]
        new_odors_palette = sns.dark_palette("r", n_colors=n_new_odors+1)[1:]
        for i in range(n_new_odors):
            lbl = new_odor_lbl + " {}".format("abcdefghijklmnop".upper()[i])
            all_colors[lbl] = new_odors_palette[i]
            ax.scatter(mixed_new_odors[i, :, dims[0]], mixed_new_odors[i, :, dims[1]], 
                        mixed_new_odors[i, :, dims[2]], s=6, lw=0.3, 
                        label=lbl+lbl_append, color=all_colors[lbl], alpha=1.0)
            if new_odor_vec is not None:
                vec = new_odor_vec[i] / l2_norm(new_odor_vec[i])
                ax.quiver(*orig, *(vec[list(dims)]), color=all_colors[lbl], 
                          lw=1.5, arrow_length_ratio=0.2)
    # No new odors shown
    else:
        n_new_odors = 0
        

    # Labeling
    for lbl, f in enumerate([ax.set_xlabel, ax.set_ylabel, ax.set_zlabel]):
        # z label gets caught in the zlbl variable at the last iteration
        zlbl = f("OSN {} (of {})".format(lbl+1, bkser.shape[1]), labelpad=-17.5)
    for f in [ax.set_xticks, ax.set_yticks, ax.set_zticks]:
        f([])
    for f in [ax.set_xticklabels, ax.set_yticklabels, ax.set_zticklabels]:
        f([], pad=0.1)
    view_params.setdefault("azim", 240)
    view_params.setdefault("elev", 3)
    ax.view_init(**view_params)
    handles, labels = ax.get_legend_handles_labels()
    # Move the label for 2+ odors to before the new odors
    if n_new_odors > 0:
        handles.insert(-n_new_odors-1, handles[0])
        labels.insert(-n_new_odors-1, labels[0])
    else:
        handles.append(handles[0])
        labels.append(labels[0])
    handles.pop(0)
    labels.pop(0)
    leg = ax.legend(handles=handles, labels=labels, 
        frameon=True, ncol=1, loc="upper left", bbox_to_anchor=(0.85, 1.0), 
        title="Odor presence", title_fontsize=6)
    #loc="upper right", bbox_to_anchor=(0.0, 1.0), frameon=False)
    fig.tight_layout()

    # Need to adjust the tightbox to remove whitespace above and below manually. 
    #ax.set_aspect("equal")
    fig.tight_layout()
    tightbox = fig.get_tightbbox()
    tightbox._bbox.y0 = tightbox._bbox.y0*1.1   #bottom
    tightbox._bbox.y1 = tightbox._bbox.y1 + 0.7*tightbox._bbox.y0  # top
    tightbox._bbox.x0 = tightbox._bbox.x0 * 0.6  # position of left side

    return fig, ax, tightbox

In [ ]:
# Inspect the background process
def pairplots_background(bkser, bkvecs, mixed_new_odors=None, new_odor_vec=None):
    # Background vectors time series with mixed concentrations
    tslice = slice(0, 50000, 20)
    vecs = bkvecs / l2_norm(bkvecs, axis=1)[:, None]
    n_comp = bkvecs.shape[0]
    n_cols = 6
    n_plots = 48 // 2
    n_rows = n_plots // n_cols + min(1, n_plots % n_cols)
    fig, axes = plt.subplots(n_rows, n_cols, sharex=True, sharey=True)
    fig.set_size_inches(n_cols*1.0, n_rows*1.0)
    single_odor_colors = sns.color_palette("colorblind", n_colors=n_comp)
    all_colors = {"Odor {}".format(i): single_odor_colors[i] for i in range(n_comp)}
    if mixed_new_odors is not None:
        n_new_odors = mixed_new_odors.shape[0]
        new_odors_palette = sns.dark_palette("r", n_colors=n_new_odors+1)[1:]
    for i in range(n_plots):
        ax = axes.flat[i]
        ax.scatter(bkser[tslice, 2*i+1], bkser[tslice, 2*i], 
                   s=9, alpha=0.5, color="k")
        for j in range(n_comp):
            ax.plot(*zip([0.0, 0.0], vecs[j, 2*i:2*i+2][::-1]), lw=2.0, 
                    color=single_odor_colors[j])
        if mixed_new_odors is not None:
            for j in range(n_new_odors):
                clr = new_odors_palette[j]
                ax.scatter(mixed_new_odors[j, :, 2*i+1], mixed_new_odors[j, :, 2*i], 
                       s=6, alpha=1.0, color=clr)
        if new_odor_vec is not None:
            for j in range(n_new_odors):
                vec = new_odor_vec[j] / l2_norm(new_odor_vec[j])
                ax.plot(*zip([0.0, 0.0], vec[2*i:2*i+2][::-1]), lw=2.0, 
                    color=new_odors_palette[j])
        ax.set(xlabel="OSN {}".format(2*i+2), ylabel="OSN {}".format(2*i+1))
    for i in range(n_plots, n_rows*n_cols):
        axes.flat[i].set_axis_off()
    fig.tight_layout()
    
    return fig, axes

In [ ]:
# Extract saved simulation
with np.load(pj(data_folder_nl, "saved_ibcm_simulations_nonlin_osn.npz")) as fp:
    ibcm_simul = load_ibcm_epsil_simul(fp, "6.0")
with np.load(pj(data_folder_nl, "saved_biopca_simulations_nonlin_osn.npz")) as fp:
    biopca_simul = load_biopca_epsil_simul(fp, "6.0")

In [ ]:
fig, ax, box = plot_manifold(ibcm_simul["bkvec_ser"], ibcm_simul["back_components"], 
                    ibcm_simul["conc_ser"], {"elev":30, "azim":210}, dims=(14, 15, 18))
#                    mixed_new_odors=ibcm_simul["mixed_new_odors"][:1], 
#                    new_odor_vec=ibcm_simul["new_odors"][:1])
if do_save_plots:
    fig.savefig(pj(panels_folder, "figure_nl_osn_manifold_moderate.pdf"), 
                transparent=True, bbox_inches=box)
plt.show()
plt.close()

In [ ]:
fig, axes = pairplots_background(ibcm_simul["bkvec_ser"][::2], ibcm_simul["back_components"])
#                                 mixed_new_odors=ibcm_simul["mixed_new_odors"][:1], 
#                                 new_odor_vec=ibcm_simul["new_odors"][:1])
if do_save_plots:
    fig.savefig(pj(panels_folder, "reviewer_figure_nonlinear_manifold_pairplot_moderate.pdf"), 
                transparent=True, bbox_inches=box)
plt.show()
plt.close()

# Panel D: IBCM convergence for moderate nonlinearity

## IBCM plotting functions

In [ ]:
def plot_ibcm_cgammas_series(t_axis, i_highlights, cgammaser):
    # Show three neurons
    fig = plt.figure()

    gs = fig.add_gridspec(3, 3)
    fig.set_size_inches(plt.rcParams["figure.figsize"][0]*0.65, 
                        plt.rcParams["figure.figsize"][1])
    ax = fig.add_subplot(gs[:, :3])
    axi = None

    #ax.axhline(0.0, ls="-", color=(0.8,)*3, lw=0.8)
    legend_styles = [[0,]*6, [0,]*6, [0,]*6]
    neuron_colors3 = neuron_colors_full[[8, 17, 23]]
    clr_back = back_palette[-1]
    plot_skp = 20
    n_b = cgammaser.shape[2]

    # plot all other neurons first, skip some points
    for i in range(n_i_ibcm):
        if i in i_highlights: 
            continue
        elif i % 2 == 0:   # thinning
            continue
        else: 
            for j in range(n_b):
                ax.plot(t_axis[::plot_skp], cgammaser[::plot_skp, i, j], color=clr_back, 
                    ls="-", alpha=1.0-0.1*j, lw=plt.rcParams["lines.linewidth"]-j*0.1)

    # Now plot the highlighted neuron
    for j in range(n_b):
        for i in range(len(i_highlights)):
            li, = ax.plot(t_axis[::plot_skp], cgammaser[::plot_skp, i_highlights[i], j], 
                          color=neuron_colors3[i], ls="-", alpha=1.0-0.1*j, 
                          lw=plt.rcParams["lines.linewidth"]-j*0.1)
            legend_styles[i][j] = li
    
    # Annotations
    ax.set(xlabel="Time (min)", 
           ylabel=r"Alignments $\bar{h}_{i\gamma} = \mathbf{\bar{m}}_i \cdot \mathbf{s}_{\gamma}$")

    gs.tight_layout(fig, w_pad=-0.2)
    return fig, ax, axi

In [ ]:
def plot_ibcm_cgammas_matrix(cgammas_mat, i_high):
    n_i, n_comp = cgammas_mat.shape
    neuron_colors3 = neuron_colors_full[[8, 17, 23]]
    
    fig, ax = plt.subplots()
    fig.set_size_inches(plt.rcParams["figure.figsize"][0]*0.4, plt.rcParams["figure.figsize"][1])
    # Extent: left, right, bottom, top
    # Greyscale version
    #ax.imshow(cgammas_matrix, cmap="Greys", aspect=0.6, extent=(0.5, n_comp+0.5, 0.5, n_i))
    #ax.set_xticks(list(range(1, n_comp+1)))
    # Colorful version: add patches manually with fill_between. 
    # Color highlighted neurons, leave others grayscale!
    normed_matrix = (cgammas_mat - cgammas_mat.min()) / (cgammas_mat.max() - cgammas_mat.min())
    for i in range(n_i):
        # Full rainbow version
        #cmap = sns.light_palette(neuron_colors_full[i], as_cmap=True)
        # Version where only highlights are colored
        if i in i_high:
            cmap = sns.light_palette(neuron_colors3[i_high.index(i)], as_cmap=True)
        else:
            cmap = sns.color_palette("Greys", as_cmap=True)
        for j in range(n_comp):
            ax.fill_between([-0.5+j, 0.5+j], -0.5+i, 0.5+i, color=cmap(normed_matrix[i, j]))

    ax.set_xlim([-0.6, -0.6+n_comp])
    ax.set_ylim([-0.6, -0.6+n_i])
    ax.set_xticks([0, 3, 5])
    ax.set_yticks(list(range(0, 19, 2)) + [21, 23])

    for i, lbl in enumerate(ax.get_yticklabels()):
        if int(lbl.get_text()) in i_high:
            clr = neuron_colors3[i_high.index(int(lbl.get_text()))]
            lbl.set_color(clr)
            ax.yaxis.get_ticklines()[i].set_color(clr)

    ax.set_xlabel(r"Component $\gamma$", size=6)
    ax.set_ylabel(r"IBCM neuron index $i$", size=6)
    cbar = fig.colorbar(mpl.cm.ScalarMappable(
        norm=mpl.colors.Normalize(cgammas_mat.min(), cgammas_mat.max()), 
        cmap="Greys"), ax=ax, aspect=30, pad=0.1)
    cbar.set_ticks([])
    cbar.set_label(label=r"Alignments ${\bar{h}}_{i\gamma}$, 45 min", fontsize=6)
    fig.tight_layout()
    return fig, ax, cbar


## IBCM plot

In [ ]:
tu_scale = 1.0 / 100.0 / 60.0  # 10 ms steps to min
cgammaser = ibcm_simul["cbars_gamma_ser"]
n_i_ibcm = cgammaser.shape[1]
nsteps = cgammaser.shape[0]
n_components = cgammaser.shape[2]
tser_example = np.arange(0.0, 360000.0, 360000 / nsteps) * tu_scale  # in min

transient = int(3*tser_example.size/4)
i_highlights = [2, 8, 21]  # Neurons to highlight

cgammas_matrix = np.mean(cgammaser[transient:], axis=0)

In [ ]:
fig, ax, axi = plot_ibcm_cgammas_series(tser_example, i_highlights, cgammaser)
if do_save_plots:
    fig.savefig(pj(panels_folder, "figure_nl_osn_ibcm_cgamma_series_moderate.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, ax, axi = plot_ibcm_cgammas_matrix(cgammas_matrix, i_highlights)
if do_save_plots:
    fig.savefig(pj(panels_folder, "figure_nl_osn_ibcm_cgamma_matrix_moderate.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel E: BioPCA convergence for moderate nonlinearity

In [ ]:
true_pca_vals = biopca_simul["true_pca_vals"]
learnt_pca_vals = biopca_simul["learnt_pca_vals"]
pca_align_error = biopca_simul["pca_align_error"]

In [ ]:
def plot_biopca_convergence(t_axis, true_pvs, learnt_pvs):
    # First plot: eigenvalues
    n_comp = learnt_pvs.shape[1]
    pca_palette = sns.color_palette("colorblind", n_colors=n_comp)
    fig, ax = plt.subplots()
    fig.set_size_inches(plt.rcParams["figure.figsize"][0]*0.95, 
                        plt.rcParams["figure.figsize"][1])

    for i in range(n_comp):
        li, = ax.plot(t_axis, learnt_pvs[:, i], label="Value {}".format(i),
                      lw=plt.rcParams["lines.linewidth"] - 0.5*i/n_comp, zorder=10-i, color=pca_palette[i])
        if true_pvs[i] / true_pvs.max() > 1e-12:
            ax.axhline(true_pvs[i], ls="--", color=pca_palette[i], 
                       lw=plt.rcParams["lines.linewidth"] - 0.5*i/n_comp, zorder=n_comp-i)
    ax.set(ylabel="Principal values, diag$(L^{-1})$", yscale="log", xlabel="Time (min)")
    # TODO: custom legend to indicate analytical vs learnt?
    handles = [mpl.lines.Line2D([0], [0], color="grey", ls="-", label=r"BioPCA ($L^{-1}$ diagonal)", 
                                lw=plt.rcParams["lines.linewidth"]), 
              mpl.lines.Line2D([0], [0], color="grey", ls="--", label="True PCA", 
                              lw=plt.rcParams["lines.linewidth"])]
    leg = ax.legend(handles=handles, frameon=False, fontsize=5.0)
    leg.set_zorder(30)
    ax.set_ylim([ax.get_ylim()[0]*0.8, ax.get_ylim()[1]])

    fig.tight_layout()
    return fig, ax

In [ ]:
# TODO: only panel missing is BioPCA!
fig, ax = plot_biopca_convergence(tser_example, true_pca_vals, learnt_pca_vals)

if do_save_plots:
    fig.savefig(os.path.join(panels_folder, "figure_nl_osn_biopca_eigenvalues_moderate.pdf"), 
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel F: performance vs nonlinearity at one concentration
To fit in the main figure, we show one concentration, we keep full tests for supplementary, they look similar anyways. 

In [ ]:
# Load saved statistics
all_jacs_stats = pd.read_hdf(os.path.join(data_folder_nl,
        "jaccard_similarities_stats_nl_osn.h5"), key="df")
all_dists_stats = pd.read_hdf(os.path.join(data_folder_nl,
        "new_mix_distances_stats_nl_osn_identity.h5"), key="df")

In [ ]:
stat_choice = "mean"
average_conc = np.sort(all_jacs_stats.index.get_level_values("new_conc").unique())[1]
n_new_concs = 1
# We want 0.5<c> for the main figure, most impressive to pick up new odors at low concentration. 
keep_conc = np.sort(all_jacs_stats.index.get_level_values("new_conc").unique())[0:n_new_concs]
epsil_range = np.sort(all_jacs_stats.index.get_level_values("epsilon").unique())
fig, axes = plt.subplots(1, n_new_concs, sharex=True, sharey=True)
if n_new_concs == 1: axes = [axes]
else: axes = axes.flatten()
width_factor = 1.0 if n_new_concs == 1 else 0.8*n_new_concs
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*width_factor, plt.rcParams["figure.figsize"][1])

# Order models according to the line order (best first)
show_models = ["optimal", "ibcm", "biopca", "avgsub", "none"]
model_zorder = ["none", "avgsub", "optimal",  "biopca", "ibcm"]
model_linestyles = {show_models[i]:neuron_styles[i % 6] for i in range(len(show_models))}
model_linestyles["ibcm"], model_linestyles["optimal"] = "-", model_linestyles["ibcm"]
for m in show_models[::-1]:  # Plot IBCM last
    if m == "none": continue  # clouds the graph too much
    for i in range(n_new_concs):
        new_conc = keep_conc[i]
        lower = (all_jacs_stats.loc[(m, epsil_range, new_conc), stat_choice] 
                 - np.sqrt(all_jacs_stats.loc[(m, epsil_range, new_conc), "var"])).clip(lower=0.0)
        upper = (all_jacs_stats.loc[(m, epsil_range, new_conc), stat_choice] 
                 + np.sqrt(all_jacs_stats.loc[(m, epsil_range, new_conc), "var"])).clip(upper=1.0)
        axes[i].fill_between(epsil_range, lower, upper, color=model_colors.get(m), alpha=0.15)
for m in show_models:
    for i in range(n_new_concs):
        new_conc = keep_conc[i]
        axes[i].plot(epsil_range, all_jacs_stats.loc[(m, epsil_range, new_conc), stat_choice], 
            label=model_nice_names.get(m, m), color=model_colors.get(m), alpha=1.0, 
            ls=model_linestyles[m], zorder=model_zorder.index(m) + 20, lw=1.25
        )
# Labeling the graphs, adding similarity between random odors, etc.
arrowprops = {"color":"k", "width":0.4, "headwidth":3.0, "headlength": 2.5}
for i in range(n_new_concs):
    axes[i].set_title(r"New conc. $= {:.1f} \langle c \rangle$".format(keep_conc[i]/average_conc), 
                     y=0.9)
    #axes[i].set_xlabel("Nonlinearity, " + r"$-\epsilon$ ($k_\mathrm{B} T$)")
    axes[i].set_xlabel(r"Log threshold, $\epsilon$ ($k_\mathrm{B} T$)" + "\n" + "Decreasing nonlinearity")
    ylim = axes[i].get_ylim()
    axes[i].set_ylim([ylim[0], 0.85])
    axes[i].axvline(3.5, ls="--", color="k", lw=0.5, ymax=0.8, zorder=100)
    axes[i].annotate(r"Sup.Fig.", xy=(3.75, 0.7), ha="center", va="bottom", fontsize=6)
    axes[i].axvline(6.0, ls="--", color="k", lw=0.5, ymax=0.8, zorder=100)
    axes[i].annotate(r"A-C", xy=(6.0, 0.7), ha="center", va="bottom", fontsize=6)
    ydip = all_jacs_stats.loc[("ibcm", 10.0, keep_conc[i]), stat_choice]
    axes[i].annotate("*", xy=(9.8, ydip-0.035), ha="left", va="top", color="k", fontsize=8)
    #axes[i].annotate("K", xy=(9.5, ydip-0.1), ha="left", va="top", color="r")
    axes[i].annotate("", xytext=(0.2, 0.055), xy=(0.65, 0.055), 
                     xycoords="figure fraction", ha="left", va="bottom", arrowprops=arrowprops)
axes[0].set_ylabel("{} Jaccard sim.".format(stat_choice.capitalize()))
leg_title = "Model"
axes[-1].legend(loc="upper left", bbox_to_anchor=(0.98, 1.), frameon=False, 
                title=leg_title, borderaxespad=0.0, handlelength=1.5)
fig.tight_layout(w_pad=0.1)
if do_save_plots:
    fig.savefig(os.path.join(panels_folder, "figure_nl_osn_jaccards_vs_epsilon.pdf"),
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Supplementary figure panels

# Panel A (former G): curved manifold at strong nonlinearity ($\epsilon = 3.5$)

In [ ]:
# Extract saved simulation
epsil_strong = 3.5
with np.load(pj(data_folder_nl, "saved_ibcm_simulations_nonlin_osn.npz")) as fp:
    ibcm_simul2 = load_ibcm_epsil_simul(fp, str(epsil_strong))
with np.load(pj(data_folder_nl, "saved_biopca_simulations_nonlin_osn.npz")) as fp:
    biopca_simul2 = load_biopca_epsil_simul(fp, str(epsil_strong))


In [ ]:
fig, ax, box = plot_manifold(
    ibcm_simul2["bkvec_ser"], 
    ibcm_simul2["back_components"][[0, 3, 5, 1, 4, 2], :], 
    ibcm_simul2["conc_ser"][:,[0, 3, 5, 1, 4, 2]], 
    {"elev":30, "azim":220}, dims=(38, 39, 40)) 
#                mixed_new_odors=ibcm_simul2["mixed_new_odors"][:1], 
#                new_odor_vec=ibcm_simul2["new_odors"][:1])
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_nl_osn_manifold_strong.pdf"), 
                transparent=True, bbox_inches=box)
plt.show()
plt.close()

In [ ]:
# Background pairplot
fig, axes = pairplots_background(ibcm_simul2["bkvec_ser"][::2], ibcm_simul2["back_components"]) 
#                                mixed_new_odors=ibcm_simul2["mixed_new_odors"][:1], 
#                                new_odor_vec=ibcm_simul2["new_odors"][:1])
if do_save_plots:
    fig.savefig(pj(panels_folder, "reviewer_figure_nonlinear_manifold_pairplot_strong.pdf"), 
                transparent=True, bbox_inches=box)
plt.show()
plt.close()

# Panel B (former H): IBCM on strongly nonlinear background
Less pretty

In [ ]:
tu_scale = 1.0 / 100.0 / 60.0  # 10 ms steps to min
cgammaser2 = ibcm_simul2["cbars_gamma_ser"]
n_i_ibcm = cgammaser2.shape[1]
nsteps = cgammaser2.shape[0]
n_components = cgammaser2.shape[2]
tser_example2 = np.arange(0.0, 360000.0, 360000 / nsteps) * tu_scale  # in min

transient = int(3*tser_example.size/4)
i_highlights2 = [6, 14, 21]  # Neurons to highlight

cgammas_matrix2 = np.mean(cgammaser2[transient:], axis=0)

In [ ]:
fig, ax, axi = plot_ibcm_cgammas_series(tser_example2, i_highlights2, cgammaser2)
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_nl_osn_ibcm_cgamma_series_strong.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, ax, axi = plot_ibcm_cgammas_matrix(cgammas_matrix2, i_highlights2)
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_nl_osn_ibcm_cgamma_matrix_strong.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel C (former I): Response to background after habituation
To show that with strong nonlinearity, part of the background is simply not inhibited by linear methods. 

Mention in caption that BioPCA would be similar to previous one, it finds extra principal components from curved regions. 

Include optimal matrix?

In [ ]:
ynormser_optimal = np.load(pj(data_folder_nl, 
        "saved_optimal_habituation_nonlin_osn.npz"))["ynorm_ser_" + str(epsil_strong)]
ynormser_optimal

In [ ]:
def plot_background_norm_inhibition(ts, bks_norm, ssers, plot_skp=4):
    """ Plot the norm of the ORN activity vector
    (background before inhibition) and of the PN activity vector
    (background after inhibition) over time to illustrate
    noise reduction by the inhibitory network.

    Let the user add analytical values and a legend.

    Args:
        ts (np.ndarray): time points
        bks (np.ndarray): time series of background input vectors norm,
            indexed [time].
        ssers (dict np.ndarray): time series of PN norm during habituation
            indexed [time], for each model
        skp (int): plot only every skp time point

    Returns:
        fig, ax
        bks_norm (np.ndarray): time series of activity
            vector norm before inhibition
        ss_norm (np.ndarray): time series of activity
            vector norm after inhibition
    """

    # Plot both norms on the same graph
    fig, ax = plt.subplots()
    fig.set_size_inches(plt.rcParams["figure.figsize"][0]*0.9, plt.rcParams["figure.figsize"][1])
    lstyles = {
        "none": "-.", 
        "ibcm": "-", 
        "biopca": "-",
        "optimal": ":"
    }
    ax.plot(ts[::plot_skp], bks_norm[::plot_skp], lw=1.0, alpha=0.8,
            color=back_palette[-1], label="No habituation")
    for mod in ssers:
        ax.plot(ts[::plot_skp], ssers[mod][::plot_skp], lw=1.0, alpha=0.8, ls=lstyles[mod],
            color=model_colors[mod], label=model_nice_names[mod])
    ax.set(xlabel="Time (min)", ylabel="PN vector norm")
    
    ax.legend(loc="upper right")
    fig.tight_layout()

    return fig, ax


In [ ]:
backnormser = l2_norm(ibcm_simul2["bkvec_ser"], axis=1)
ynormsers = {
    "ibcm": ibcm_simul2["y_norm_ser"],
    "biopca": biopca_simul2["y_norm_ser"],
    "optimal": ynormser_optimal
}

In [ ]:
fig, ax = plot_background_norm_inhibition(tser_example, backnormser, ynormsers)
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_nl_osn_strong_nonlinearity_habituation.pdf"), 
                transparent=True, bbox_inches="tight")

# Panel D (former J): performance vs strength of nonlinearity

In [ ]:
# Load saved statistics
all_jacs_stats = pd.read_hdf(os.path.join(data_folder_nl,
        "jaccard_similarities_stats_nl_osn.h5"), key="df")
all_dists_stats = pd.read_hdf(os.path.join(data_folder_nl,
        "new_mix_distances_stats_nl_osn_identity.h5"), key="df")

In [ ]:
stat_choice = "mean"
average_conc = np.sort(all_jacs_stats.index.get_level_values("new_conc").unique())[1]
n_new_concs = 2
keep_conc = np.sort(all_jacs_stats.index.get_level_values("new_conc").unique())[0:n_new_concs]
epsil_range = np.sort(all_jacs_stats.index.get_level_values("epsilon").unique())
fig, axes = plt.subplots(1, n_new_concs, sharex=True, sharey=True)
if n_new_concs == 1: axes = [axes]
else: axes = axes.flatten()
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.6, plt.rcParams["figure.figsize"][1])

# Order models according to the line order (best first)
show_models = ["optimal", "ibcm", "biopca", "avgsub", "none"]
model_zorder = ["none", "avgsub", "optimal",  "biopca", "ibcm"]
model_linestyles = {show_models[i]:neuron_styles[i % 6] for i in range(len(show_models))}
model_linestyles["ibcm"], model_linestyles["optimal"] = "-", model_linestyles["ibcm"]
for m in show_models[::-1]:  # Plot IBCM last
    if m == "none": continue  # clouds the graph too much
    for i in range(n_new_concs):
        new_conc = keep_conc[i]
        lower = (all_jacs_stats.loc[(m, epsil_range, new_conc), stat_choice] 
                 - np.sqrt(all_jacs_stats.loc[(m, epsil_range, new_conc), "var"])).clip(lower=0.0)
        upper = (all_jacs_stats.loc[(m, epsil_range, new_conc), stat_choice] 
                 + np.sqrt(all_jacs_stats.loc[(m, epsil_range, new_conc), "var"])).clip(upper=1.0)
        axes[i].fill_between(epsil_range, lower, upper, color=model_colors.get(m), alpha=0.15)
for m in show_models:
    for i in range(n_new_concs):
        new_conc = keep_conc[i]
        axes[i].plot(epsil_range, all_jacs_stats.loc[(m, epsil_range, new_conc), stat_choice], 
            label=model_nice_names.get(m, m), color=model_colors.get(m), alpha=1.0, 
            ls=model_linestyles[m], zorder=model_zorder.index(m) + 20, lw=1.25
        )
# Labeling the graphs, adding similarity between random odors, etc.
arrowprops = {"color":"k", "width":0.4, "headwidth":3.0, "headlength": 2.5}
for i in range(n_new_concs):
    axes[i].set_title(r"New conc. $= {:.1f} \langle c \rangle$".format(keep_conc[i]/average_conc), 
                     y=0.9)
    #axes[i].set_xlabel("Nonlinearity, " + r"$-\epsilon$ ($k_\mathrm{B} T$)")
    axes[i].set_xlabel(r"Log threshold, $\epsilon$ ($k_\mathrm{B} T$)" + "\n" + "Decreasing nonlinearity")
    ylim = axes[i].get_ylim()
    axes[i].set_ylim([ylim[0], 0.85])
    axes[i].axvline(3.5, ls="--", color="k", lw=0.5, ymax=0.8, zorder=100)
    axes[i].annotate(r"A-C", xy=(3.5, 0.7), ha="center", va="bottom", fontsize=6)
    axes[i].axvline(6.0, ls="--", color="k", lw=0.5, ymax=0.8, zorder=100)
    axes[i].annotate(r"Main Fig.", xy=(6.0, 0.7), ha="center", va="bottom", fontsize=6)
    ydip = all_jacs_stats.loc[("ibcm", 10.0, keep_conc[i]), stat_choice]
    axes[i].annotate("*", xy=(9.8, ydip-0.035), ha="left", va="top", color="r")
    axes[i].annotate("E", xy=(9.5, ydip-0.1), ha="left", va="top", color="r")
    axes[i].annotate("", xytext=(0.11 + 0.3725*i, 0.055), xy=(0.43 + 0.3725*i, 0.055), 
                     xycoords="figure fraction", ha="left", va="bottom", arrowprops=arrowprops)
axes[0].set_ylabel("{} Jaccard sim.".format(stat_choice.capitalize()))
leg_title = "Model"
axes[-1].legend(loc="upper left", bbox_to_anchor=(0.98, 1.), frameon=False, 
                title=leg_title, borderaxespad=0.0, handlelength=1.5)
fig.tight_layout(w_pad=0.1)
if do_save_plots:
    fig.savefig(os.path.join(panels_folder, "supfig_nl_osn_jaccards_vs_epsilon.pdf"),
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel E (former K): explain why IBCM fails at high $\epsilon$ (linear case)
Because it converges too fast and misses some odors, in some cases. 
Include a panel of dot products. 

In [ ]:
# load relevant results
# Extract saved simulation
with np.load(pj(data_folder_nl, "saved_ibcm_simulations_nonlin_osn.npz")) as fp:
    ibcm_simul_fail = load_ibcm_epsil_simul(fp, "10.0")
    biopca_simul_fail = load_biopca_epsil_simul(fp, "10.0")

In [ ]:
tu_scale = 1.0 / 100.0 / 60.0  # 10 ms steps to min
cgammaser_fail = ibcm_simul_fail["cbars_gamma_ser"]
n_i_ibcm = cgammaser_fail.shape[1]
nsteps = cgammaser_fail.shape[0]
n_components = cgammaser_fail.shape[2]
tser_example = np.arange(0.0, 360000.0, 360000 / nsteps) * tu_scale  # in min

transient = int(3*tser_example.size/4)
cgammas_matrix = np.mean(cgammaser_fail[transient:], axis=0)
i_highlights = [2, 6, 21]  # Neurons to highlight

In [ ]:
def plot_ibcm_cgammas_series_percomp(t_axis, comp_high, cgammaser, n_b):
    """ Assumes a separate cgammas matrix will be plotted as a legend """
    fig, ax = plt.subplots()
    fig.set_size_inches(plt.rcParams["figure.figsize"][0]*0.75, 
                        plt.rcParams["figure.figsize"][1])

    odor_colors = sns.color_palette("colorblind", n_colors=n_b)
    clr_back = back_palette[-1]
    plot_skp = 20

    # plot all other components first, skip some points
    for j in range(n_b):
        if j in comp_high: 
            continue
        else:
            for i in range(0, n_i_ibcm, 2):
                ax.plot(t_axis[::plot_skp], cgammaser[::plot_skp, i, j], color=clr_back, 
                    ls="-", alpha=0.8, lw=plt.rcParams["lines.linewidth"])

    # Now plot the highlighted components
    for j in range(len(comp_high)):
        for i in range(0, n_i_ibcm, 2):
            li, = ax.plot(t_axis[::plot_skp], cgammaser[::plot_skp, i, comp_high[j]], 
                          color=odor_colors[j], ls="-", alpha=1.0, 
                          lw=plt.rcParams["lines.linewidth"])

    ax.set(xlabel="Time (min)", 
           ylabel=r"Alignments $\bar{h}_{i\gamma} = \mathbf{\bar{m}}_i \cdot \mathbf{s}_{\gamma}$")

    fig.tight_layout()
    return fig, ax

In [ ]:
def plot_ibcm_cgammas_matrix_percomp(cgammas_mat, comp_high, n_i, n_comp):
    fig, ax = plt.subplots()
    fig.set_size_inches(plt.rcParams["figure.figsize"][0]*0.6, 
                        plt.rcParams["figure.figsize"][1])
    
    normed_matrix = (cgammas_mat - cgammas_mat.min()) / (cgammas_mat.max() - cgammas_mat.min())
    odor_colors = sns.color_palette("colorblind", n_colors=n_comp)
    for j in range(n_comp):
        if j in comp_high:
            cmap = sns.light_palette(odor_colors[comp_high.index(j)], as_cmap=True)
        else:
            cmap = sns.color_palette("Greys", as_cmap=True)
        for i in range(n_i):
            ax.fill_between([-0.5+j, 0.5+j], -0.5+i, 0.5+i, color=cmap(normed_matrix[i, j]))

    ax.set_xlim([-0.6, -0.6+n_comp])
    ax.set_ylim([-0.6, -0.6+n_i])
    ax.set_xticks(list(range(0, n_comp)))
    ax.set_yticks(list(range(0, n_i, 2)))

    for j, lbl in enumerate(ax.get_xticklabels()):
        if int(lbl.get_text()) in comp_high:
            clr = odor_colors[comp_high.index(int(lbl.get_text()))]
            lbl.set_color(clr)
            ax.xaxis.get_ticklines()[j].set_color(clr)
    ax.set(xlabel=r"Component $\gamma$", ylabel="IBCM neuron index $i$")
    cbar = fig.colorbar(mpl.cm.ScalarMappable(
        norm=mpl.colors.Normalize(cgammas_mat.min(), cgammas_mat.max()), 
        cmap="Greys"), ax=ax, label=r"Alignments ${\bar{h}}_{i\gamma}$, 45 min", aspect=30, pad=0.1)
    fig.tight_layout()
    return fig, ax, cbar


In [ ]:
# Special plotting version to highlight components, not neurons
comp_highlights = [1, 5]
fig, ax, cbar = plot_ibcm_cgammas_matrix_percomp(cgammas_matrix, comp_highlights, n_i_ibcm, n_components)

# Highlight column without a single specific neuron
missing_comp = 5
# Create a Rectangle patch. xy, width, height
rect = mpl.patches.Rectangle((missing_comp-0.5, -0.3), 0.8, n_i_ibcm-0.5, linewidth=1, 
                edgecolor='r', ls="--", facecolor='none')
# Add the patch to the Axes
ax.add_patch(rect)
if do_save_plots:
    fig.savefig(os.path.join(panels_folder, "supfig_nl_osn_cgamma_matrix_failed_ibcm.pdf"), 
            transparent=True, bbox_inches="tight", bbox_extra_artists=(cbar.ax,))
plt.show()
plt.close()

In [ ]:
# Special plotting version to highlight components, not neurons
comp_highlights = [1, 5]
fig, ax = plot_ibcm_cgammas_series_percomp(tser_example, comp_highlights, 
                                cgammaser_fail, n_components)

if do_save_plots:
        fig.savefig(os.path.join(panels_folder, "supfig_nl_osn_cgamma_series_failed_ibcm.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()